# Solutions

## About the Data
In this notebook, we will be working with two data sources: 
- 2018 stock data for Facebook, Apple, Amazon, Netflix, and Google (obtained using the [`stock_analysis`](https://github.com/fenago/stock-analysis) package)
- European Centre for Disease Prevention and Control's (ECDC) [daily number of new reported cases of COVID-19 by country worldwide dataset](https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide) collected on September 19, 2020 via [this link](https://opendata.ecdc.europa.eu/covid19/casedistribution/csv)

## Setup

In [ ]:
import pandas as pd

## Exercise 1
We want to look at data for the FAANG stocks (Facebook, Apple, Amazon, Netflix, and Google), but we were given each as a separate CSV file. Make them into a single file and store the dataframe of the FAANG data as `faang`:
1. Read each file in.
2. Add a column to each dataframe indicating the ticker it is for.
3. Append them together into a single dataframe.
4. Save the result to a CSV file.

In [ ]:
faang = pd.DataFrame()
for ticker in ['fb', 'aapl', 'amzn', 'nflx', 'goog']:
    df = pd.read_csv(f'../../lab_08/exercises/{ticker}.csv')
    # make the ticker the first column
    df.insert(0, 'ticker', ticker.upper())
    faang = faang.append(df)

faang.to_csv('faang.csv', index=False)

## Exercise 2
With `faang`, use type conversion to change the `date` column to datetime and the `volume` column to integers. Then, sort by `date` and `ticker`.

In [ ]:
faang = faang.assign(
    date=lambda x: pd.to_datetime(x.date),
    volume=lambda x: x.volume.astype(int)
).sort_values(
    ['date', 'ticker']
)

faang.head()

## Exercise 3
Find the 7 rows with the lowest value for `volume`.

In [ ]:
faang.nsmallest(7, 'volume')

## Exercise 4
Right now, the data is somewhere between long and wide format. Use `melt()` to make it completely long format.

In [ ]:
melted_faang = faang.melt(
    id_vars=['ticker', 'date'], 
    value_vars=['open', 'high', 'low', 'close', 'volume']
)
melted_faang.head()

## Exercise 5
Suppose we found out there was a glitch in how the data was recorded on July 26, 2018. How should we handle this?

> Given that this is a large data set (~ 1 year), we would be tempted to just drop that date and interpolate. However, some preliminary research on that date for the FAANG stocks reveals that FB took a huge tumble that day. If we had interpolated, we would have missed the magnitude of the drop.

## Exercise 6
The European Centre for Disease Prevention and Control (ECDC) provides an open dataset on COVID-19 cases called, [*daily number of new reported cases of COVID-19 by country worldwide*](https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide). This dataset is updated daily, but we will use a snapshot that contains data from January 1, 2020 through September 18, 2020. Clean and pivot the data so that it is in wide format:

1. Read in the `covid19_cases.csv` file.
2. Create a `date` column using the data in the `dateRep` column and the `pd.to_datetime()` function.
3. Set the `date` column as the index and sort the index.
4. Replace occurrences of `United_States_of_America` and `United_Kingdom` with `USA` and `UK`, respectively.
5. Using the `countriesAndTerritories` column, filter the data down to Argentina, Brazil, China, Colombia, India, Italy, Mexico, Peru, Russia, Spain, Turkey, the UK, and the USA.
6. Pivot the data so that the index contains the dates, the columns contain the country names, and the values are the case counts in the `cases` column. Be sure to fill in `NaN` values with `0`.

In [ ]:
covid = pd.read_csv('../../lab_08/exercises/covid19_cases.csv').assign(
    date=lambda x: pd.to_datetime(x.dateRep, format='%d/%m/%Y')
).set_index('date').replace(
    'United_States_of_America', 'USA'
).replace('United_Kingdom', 'UK').sort_index()

covid[
    covid.countriesAndTerritories.isin([
        'Argentina', 'Brazil', 'China', 'Colombia', 'India', 'Italy', 
        'Mexico', 'Peru', 'Russia', 'Spain', 'Turkey', 'UK', 'USA'
    ])
].reset_index().pivot(index='date', columns='countriesAndTerritories', values='cases').fillna(0)

## Exercise 7
In order to determine the case totals per country efficiently, we need the aggregation skills we will learn in *Lab 4, Aggregating DataFrames*, so the ECDC data in the `covid19_cases.csv` file has been aggregated for us and saved in the `covid19_total_cases.csv` file. It contains the total number of case per country. Use this data to find the 20 countries with the largest COVID-19 case totals. Hints:

- When reading in the CSV file, pass in `index_col='cases'`.
- Note that it will be helpful to transpose the data before isolating the countries.

In [ ]:
pd.read_csv('../../lab_08/exercises/covid19_total_cases.csv', index_col='index')\
    .T.nlargest(20, 'cases').sort_values('cases', ascending=False)